# Dataloader Debugging Notebook

Interactive analysis of internal vs external dataloader performance.

**Purpose**: Diagnose why internal (51.8% acc) underperforms external (69.6% acc)

**Experiments**:
- Internal: `outputs/siamese_e2e/20260113_073023`
- External: `outputs/siamese_e2e/20260111_005327`

**Sections**:
1. Training curves comparison
2. Weight snapshot analysis
3. Batch statistics
4. Holdout predictions
5. Error analysis

In [ ]:
# Imports and setup
import sys
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# Add project to path
sys.path.insert(0, str(Path.cwd().parent))

# Plotting setup
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Experiment paths
INTERNAL_EXP = Path('../outputs/siamese_e2e/20260113_073023')
EXTERNAL_EXP = Path('../outputs/siamese_e2e/20260111_005327')

print(f"Internal experiment: {INTERNAL_EXP.exists()}")
print(f"External experiment: {EXTERNAL_EXP.exists()}")

## 1. Training Curves Comparison

Compare loss and accuracy curves side-by-side.

In [ ]:
# Load training history
with open(INTERNAL_EXP / 'training_history.json') as f:
    internal_history = json.load(f)

with open(EXTERNAL_EXP / 'training_history.json') as f:
    external_history = json.load(f)

# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

epochs_internal = range(1, len(internal_history['train_loss']) + 1)
epochs_external = range(1, len(external_history['train_loss']) + 1)

# Train loss
axes[0, 0].plot(epochs_internal, internal_history['train_loss'], 'b-o', label='Internal', linewidth=2)
axes[0, 0].plot(epochs_external, external_history['train_loss'], 'r-s', label='External', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Val loss
axes[0, 1].plot(epochs_internal, internal_history['val_loss'], 'b-o', label='Internal', linewidth=2)
axes[0, 1].plot(epochs_external, external_history['val_loss'], 'r-s', label='External', linewidth=2)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Validation Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Train acc
axes[1, 0].plot(epochs_internal, internal_history['train_acc'], 'b-o', label='Internal', linewidth=2)
axes[1, 0].plot(epochs_external, external_history['train_acc'], 'r-s', label='External', linewidth=2)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy (%)')
axes[1, 0].set_title('Training Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Val acc
axes[1, 1].plot(epochs_internal, internal_history['val_acc'], 'b-o', label='Internal', linewidth=2)
axes[1, 1].plot(epochs_external, external_history['val_acc'], 'r-s', label='External', linewidth=2)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy (%)')
axes[1, 1].set_title('Validation Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=50, color='gray', linestyle='--', label='Random', alpha=0.5)

plt.tight_layout()
plt.show()

# Print final metrics
print("Final Metrics:")
print(f"  Internal: train_acc={internal_history['train_acc'][-1]:.1f}%, val_acc={internal_history['val_acc'][-1]:.1f}%")
print(f"  External: train_acc={external_history['train_acc'][-1]:.1f}%, val_acc={external_history['val_acc'][-1]:.1f}%")

## 2. Gradient Norm Comparison

Compare gradient norms to understand optimization dynamics.

In [ ]:
# Load gradient norms
grad_internal = pd.read_csv(INTERNAL_EXP / 'telemetry' / 'gradient_norms.csv')
grad_external = pd.read_csv(EXTERNAL_EXP / 'telemetry' / 'gradient_norms.csv')

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Overall gradient norm
axes[0].plot(grad_internal.index, grad_internal['overall'], 'b-', alpha=0.7, label='Internal')
axes[0].plot(grad_external.index, grad_external['overall'], 'r-', alpha=0.7, label='External')
axes[0].set_xlabel('Collection Step')
axes[0].set_ylabel('Gradient Norm')
axes[0].set_title('Overall Gradient Norm')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Backbone gradient norm
axes[1].plot(grad_internal.index, grad_internal['backbone'], 'b-', alpha=0.7, label='Internal')
axes[1].plot(grad_external.index, grad_external['backbone'], 'r-', alpha=0.7, label='External')
axes[1].set_xlabel('Collection Step')
axes[1].set_ylabel('Gradient Norm')
axes[1].set_title('Backbone Gradient Norm')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Head gradient norm
axes[2].plot(grad_internal.index, grad_internal['head'], 'b-', alpha=0.7, label='Internal')
axes[2].plot(grad_external.index, grad_external['head'], 'r-', alpha=0.7, label='External')
axes[2].set_xlabel('Collection Step')
axes[2].set_ylabel('Gradient Norm')
axes[2].set_title('Head Gradient Norm')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average gradient norms:")
print(f"  Internal: overall={grad_internal['overall'].mean():.4f}, backbone={grad_internal['backbone'].mean():.4f}, head={grad_internal['head'].mean():.4f}")
print(f"  External: overall={grad_external['overall'].mean():.4f}, backbone={grad_external['backbone'].mean():.4f}, head={grad_external['head'].mean():.4f}")

## 3. Batch Statistics

Compare winner label distribution in training batches.

In [ ]:
# Load batch stats
batch_internal = pd.read_csv(INTERNAL_EXP / 'telemetry' / 'batch_stats.csv')
batch_external = pd.read_csv(EXTERNAL_EXP / 'telemetry' / 'batch_stats.csv')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Winner=1 percentage over time
axes[0].plot(batch_internal.index, batch_internal['winner_1_pct'], 'b-', alpha=0.6, label='Internal')
axes[0].plot(batch_external.index, batch_external['winner_1_pct'], 'r-', alpha=0.6, label='External')
axes[0].axhline(y=50, color='gray', linestyle='--', label='Balanced (50%)', alpha=0.5)
axes[0].set_xlabel('Collection Step')
axes[0].set_ylabel('Winner=1 Percentage')
axes[0].set_title('Batch Label Distribution Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution histograms
axes[1].hist(batch_internal['winner_1_pct'], bins=20, alpha=0.5, color='blue', label='Internal', edgecolor='black')
axes[1].hist(batch_external['winner_1_pct'], bins=20, alpha=0.5, color='red', label='External', edgecolor='black')
axes[1].axvline(x=50, color='gray', linestyle='--', label='Balanced', alpha=0.7)
axes[1].set_xlabel('Winner=1 Percentage')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Batch Label Balance')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Winner=1 percentage statistics:")
print(f"  Internal: mean={batch_internal['winner_1_pct'].mean():.1f}%, std={batch_internal['winner_1_pct'].std():.1f}%")
print(f"  External: mean={batch_external['winner_1_pct'].mean():.1f}%, std={batch_external['winner_1_pct'].std():.1f}%")
print(f"\n⚠️ External has {batch_external['winner_1_pct'].mean() - 50:.1f}% bias toward winner=1!")

## 4. Holdout Predictions Analysis

Compare model confidence and accuracy on fixed validation subset.

In [ ]:
# Load holdout predictions
holdout_internal = pd.read_csv(INTERNAL_EXP / 'telemetry' / 'holdout_predictions.csv')
holdout_external = pd.read_csv(EXTERNAL_EXP / 'telemetry' / 'holdout_predictions.csv')

# Get final epoch predictions
final_internal = holdout_internal[holdout_internal['epoch'] == holdout_internal['epoch'].max()]
final_external = holdout_external[holdout_external['epoch'] == holdout_external['epoch'].max()]

# Compute confidence (difference between logits)
final_internal['confidence'] = (final_internal['logit_1'] - final_internal['logit_0']).abs()
final_external['confidence'] = (final_external['logit_1'] - final_external['logit_0']).abs()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Accuracy over time
acc_internal = holdout_internal.groupby('batch_idx')['correct'].mean() * 100
acc_external = holdout_external.groupby('batch_idx')['correct'].mean() * 100

axes[0, 0].plot(acc_internal.index, acc_internal.values, 'b-o', label='Internal', linewidth=2, markersize=4)
axes[0, 0].plot(acc_external.index, acc_external.values, 'r-s', label='External', linewidth=2, markersize=4)
axes[0, 0].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
axes[0, 0].set_xlabel('Batch Index')
axes[0, 0].set_ylabel('Accuracy (%)')
axes[0, 0].set_title('Holdout Accuracy Over Training')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Confidence histograms
axes[0, 1].hist(final_internal['confidence'], bins=20, alpha=0.5, color='blue', label='Internal', edgecolor='black')
axes[0, 1].hist(final_external['confidence'], bins=20, alpha=0.5, color='red', label='External', edgecolor='black')
axes[0, 1].set_xlabel('Confidence (|logit1 - logit0|)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Prediction Confidence (Final Epoch)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Confidence vs correctness
axes[1, 0].scatter(final_internal['confidence'], final_internal['correct'], alpha=0.5, c='blue', label='Internal', s=50)
axes[1, 0].scatter(final_external['confidence'], final_external['correct'], alpha=0.5, c='red', label='External', s=50, marker='s')
axes[1, 0].set_xlabel('Confidence')
axes[1, 0].set_ylabel('Correct (0 or 1)')
axes[1, 0].set_title('Confidence vs Correctness')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Label distribution in holdout
internal_label_dist = final_internal['label'].value_counts()
external_label_dist = final_external['label'].value_counts()

x = [0, 1]
axes[1, 1].bar([i - 0.2 for i in x], [internal_label_dist.get(i, 0) for i in x], width=0.4, color='blue', alpha=0.7, label='Internal Labels')
axes[1, 1].bar([i + 0.2 for i in x], [external_label_dist.get(i, 0) for i in x], width=0.4, color='red', alpha=0.7, label='External Labels')
axes[1, 1].set_xlabel('Label (0 or 1)')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Holdout Label Distribution')
axes[1, 1].set_xticks(x)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final holdout metrics:")
print(f"  Internal: accuracy={final_internal['correct'].mean()*100:.1f}%, avg_confidence={final_internal['confidence'].mean():.3f}")
print(f"  External: accuracy={final_external['correct'].mean()*100:.1f}%, avg_confidence={final_external['confidence'].mean():.3f}")

## 5. Summary & Conclusions

Key findings from the analysis.

In [ ]:
print("="*80)
print("SUMMARY OF FINDINGS")
print("="*80)

print("\n1. TRAINING PERFORMANCE:")
print(f"   Internal: {internal_history['val_acc'][-1]:.1f}% validation accuracy")
print(f"   External: {external_history['val_acc'][-1]:.1f}% validation accuracy")
print(f"   Gap: {external_history['val_acc'][-1] - internal_history['val_acc'][-1]:.1f} percentage points")

print("\n2. LABEL DISTRIBUTION BIAS:")
internal_bias = batch_internal['winner_1_pct'].mean() - 50
external_bias = batch_external['winner_1_pct'].mean() - 50
print(f"   Internal: {internal_bias:+.1f}% bias (mean winner=1: {batch_internal['winner_1_pct'].mean():.1f}%)")
print(f"   External: {external_bias:+.1f}% bias (mean winner=1: {batch_external['winner_1_pct'].mean():.1f}%)")

if abs(external_bias) > 10:
    print(f"   ⚠️ External has SIGNIFICANT {external_bias:+.1f}% bias toward winner=1!")
    print(f"   This makes the learning task easier and inflates accuracy.")

print("\n3. GRADIENT NORMS:")
print(f"   Internal: {grad_internal['overall'].mean():.4f} (overall)")
print(f"   External: {grad_external['overall'].mean():.4f} (overall)")
print(f"   Difference: {abs(grad_internal['overall'].mean() - grad_external['overall'].mean()):.4f}")

print("\n4. MODEL CONFIDENCE:")
print(f"   Internal: {final_internal['confidence'].mean():.3f} (avg confidence)")
print(f"   External: {final_external['confidence'].mean():.3f} (avg confidence)")

print("\n" + "="*80)
print("CONCLUSION:")
print("="*80)

if abs(external_bias) > 10:
    print("\n🔴 ROOT CAUSE: External dataloader has BIASED labels!")
    print(f"   External has {external_bias:+.1f}% bias toward winner=1")
    print(f"   This makes it easier to achieve high accuracy by learning the bias")
    print(f"   rather than actual image quality differences.")
    print("\n   Internal uses balanced data → harder task → appears to underperform")
    print("   But internal is actually learning the CORRECT task!")
else:
    print("\n🟡 Labels appear balanced. Further investigation needed:")
    print("   - Check transform differences")
    print("   - Analyze weight evolution")
    print("   - Compare error patterns")

print("\n" + "="*80)